# Setup

### Model decisions

In [1]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
google_colab = False
hyperparameter_search = True
training_size = 991232
random_seed = 0
checkpoint_time = None # Insert checkpoint time if continuing training
starting_window = 0 # Insert starting window if continuing training
run_training = True
run_shap = False
run_prediction = False

In [28]:
if google_colab == True:
  from google.colab import drive
  drive.mount('/content/drive')

### Imports

In [29]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.math import multiply
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from math import log
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.metrics import mean_squared_error
import shap
from pathlib import Path
import tensorflow as tf

In [30]:
# If running in colab, insert your wandb key here

#import config
#Erlend
wandb.login(key='3cae81eb56be3190be5bb48c571e69933071df69')
# Hjalmar
#wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load, split and normalize data

### Load data

In [31]:
if google_colab:
    import tensorflow as tf
    # Print info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1Doyuyo_VDOmJf0CLo5kl9XzMTfhGtxiR"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2010-2023_NSS_filtered_vF.csv')  
    df_read = pd.read_csv('2010-2023_NSS_filtered_vF.csv')
else:
    file = "../data/processed_data/2010-2023_NSS_filtered_vF.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R
0,0,2010-01-04,207.490,1132.99,925.0,0.008219,0.00050
1,1,2010-01-04,182.500,1132.99,950.0,0.008219,0.00050
2,2,2010-01-04,157.500,1132.99,975.0,0.008219,0.00050
3,3,2010-01-04,132.600,1132.99,1000.0,0.008219,0.00050
4,4,2010-01-04,107.705,1132.99,1025.0,0.008219,0.00050
...,...,...,...,...,...,...,...
12057638,13739049,2023-03-31,217.750,4109.88,4700.0,1.726027,0.04198
12057639,13739050,2023-03-31,180.000,4109.88,4800.0,1.726027,0.04198
12057640,13739051,2023-03-31,146.550,4109.88,4900.0,1.726027,0.04198
12057641,13739052,2023-03-31,118.200,4109.88,5000.0,1.726027,0.04198


### Calculate volatility

In [32]:
df = df_read
del df_read

if hyperparameter_search:
    df = df[df['Quote_date'] <= '2015-02-01']

if run_shap:
    df = df[df['Quote_date'] >= '2019-01-01']

# Add volatility column with 30 day rolling standard deviation of Underlying_last

# New dataframe without duplicate Quote_dates
df2 = df.drop_duplicates(subset=['Quote_date'])

# Calculate volatility
df2['Volatility'] = np.log(df2["Underlying_last"] / df2["Underlying_last"].shift()).rolling(30).std()*(252**0.5)

# Matching volatility in df2 to df
df['Volatility'] = df['Quote_date'].map(df2.set_index('Quote_date')['Volatility'])

# Filter df between 2011-09-01
df = df[(df["Quote_date"] >= "2011-09-01")] if hyperparameter_search else df[(df["Quote_date"] >= "2011-12-01")]

print(df)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


          Unnamed: 0  Quote_date    Price  Underlying_last  Strike       TTM   
342523        446309  2011-12-01  344.545          1244.09   900.0  0.002740  \
342524        446310  2011-12-01  319.695          1244.09   925.0  0.002740   
342525        446311  2011-12-01  294.595          1244.09   950.0  0.002740   
342526        446312  2011-12-01  269.610          1244.09   975.0  0.002740   
342527        446313  2011-12-01  244.600          1244.09  1000.0  0.002740   
...              ...         ...      ...              ...     ...       ...   
12057638    13739049  2023-03-31  217.750          4109.88  4700.0  1.726027   
12057639    13739050  2023-03-31  180.000          4109.88  4800.0  1.726027   
12057640    13739051  2023-03-31  146.550          4109.88  4900.0  1.726027   
12057641    13739052  2023-03-31  118.200          4109.88  5000.0  1.726027   
12057642    13739053  2023-03-31   94.400          4109.88  5100.0  1.726027   

                R  Volatility  
342523 

### Format input data

In [40]:
# Format settings
bs_vars = ['Underlying_last', 'Strike', 'TTM', 'R', 'Volatility']

def create_rw_dataset(window_number = 0, df = None, shap = False):
    '''Creates dataset for a single rolling window period offsett by the window number'''

    # Create train, validation and test set split points
    test_months = 1
    train_start = datetime(2011,11,1) + relativedelta(months=window_number * test_months) if hyperparameter_search else datetime(2011,12,1) + relativedelta(months=window_number * test_months)
    val_start = train_start + relativedelta(months=3*12)
    test_start = val_start + relativedelta(months = 1 + test_months) if hyperparameter_search else val_start + relativedelta(months = 1)
    test_end = test_start + relativedelta(months=test_months)
    train_start = str(train_start.date())
    val_start = str(val_start.date())
    test_start = str(test_start.date())
    test_end = str(test_end.date())
        
    # Split train and validation data
    df_train = df[(df['Quote_date'] >= train_start) & (df['Quote_date'] < val_start)]
    df_val = df[(df['Quote_date'] >= val_start) & (df['Quote_date'] < test_start)]
    df_test = df[(df['Quote_date'] >= test_start) & (df['Quote_date'] < test_end)]

    del df

    # Extract target values
    train_y = df_train['Price'].to_numpy()
    val_y = df_val['Price'].to_numpy()
    test_y = df_test['Price'].to_numpy()

    # Print earliest and latest date in every dataframe used
    print("--------------Dataframe dates--------------")
    print(f"Train: {df_train['Quote_date'].min()} - {df_train['Quote_date'].max()}")
    print(f"Val: {df_val['Quote_date'].min()} - {df_val['Quote_date'].max()}")
    print(f"Test: {df_test['Quote_date'].min()} - {df_test['Quote_date'].max()}")
    print("-------------------------------------------")

    # Convert dataframes to numpy arrays
    train_x = df_train[bs_vars].to_numpy()
    val_x = df_val[bs_vars].to_numpy()
    test_x = df_test[bs_vars].to_numpy()

    del df_train
    del df_val

    # Scale features based on training set
    bs_scaler = MinMaxScaler()
    train_x = bs_scaler.fit_transform(train_x)
    val_x = bs_scaler.transform(val_x)
    test_x = bs_scaler.transform(test_x)


    # Shuffle training set
    np.random.seed(random_seed)
    shuffle = np.random.permutation(len(train_x))
    train_x = train_x[shuffle]
    train_y = train_y[shuffle]

    # Extract training set
    train_x = train_x[:training_size]
    train_y = train_y[:training_size]
    
    if shap:
        return train_x, test_x, train_start, val_start, test_start

    print(f'Train shape: {train_x.shape},')
    print(f'Val shape: {val_x.shape},')
    print(f'Test shape: {test_x.shape},')

    return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test

# Create the dataset for the first rolling window period
if not run_shap:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df)
    if hyperparameter_search:
        del df

--------------Dataframe dates--------------
Train: 2011-12-01 - 2014-10-31
Val: 2014-11-03 - 2014-12-31
Test: 2015-01-02 - 2015-01-30
-------------------------------------------
Train shape: (991232, 5),
Val shape: (115455, 5),
Test shape: (56797, 5),


# Model construction

In [34]:
def create_model(config):
    '''Builds a MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    bs_vars = Input((config.Num_features,))

    # MLP layers
    layers = Dense(config.MLP_units)(bs_vars)
    layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
    layers = LeakyReLU(alpha=config.Leaky_relu_alpha)(layers)
    
    for _ in range(config.MLP_layers-1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU(alpha=config.Leaky_relu_alpha)(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x)/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=bs_vars, outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Help functions

In [35]:
import gc
from tensorflow.keras import backend as k

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

## Creating trainer function

In [36]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None, checkpoint_path = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )
        
        # Check if the checkpoint folder exists
        if checkpoint_path and not os.path.exists(checkpoint_path):
            # Create the checkpoint folder if it does not exist
            os.makedirs(checkpoint_path)
        
        if checkpoint_path:
          checkpoint = ModelCheckpoint(
              filepath=checkpoint_path + ".h5",
              monitor='val_loss',
              mode='min',
              save_best_only=True,
              save_weights_only=False
          )

        print(f'Train shape: {train_x.shape}')
        print(f'Val shape: {val_x.shape}')

        # Train model
        model.fit(
            train_x,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback, checkpoint, ClearMemory()] if checkpoint_path else [early_stopping, wandb_callback, ClearMemory()],
        )


# Hyperparameter search

In [41]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'MLP v1.0',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'MLP_units': {
            'values': [100, 200, 400, 600]},        
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 7, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'uniform',
            'max': 0.1, 'min': 0.001},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.8},
        'Leaky_relu_alpha': {
            'values': [0.01, 0.05, 0.1, 0.3]},
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value':  5},
        'Training_size': {
            'value': training_size},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - MLP') 
#sweep_id = 'kdb4nbpi'

if hyperparameter_search:
    wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - MLP', count = 1000)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: vony09oh
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20MLP/sweeps/vony09oh


wandb: Agent Starting Run: ey4rbj11 with config:
wandb: 	Bn_momentum: 0.99
wandb: 	Leaky_relu_alpha: 0.3
wandb: 	Lr: 0.0002872795103650739
wandb: 	Lr_decay: 0.8408169239701624
wandb: 	MLP_layers: 2
wandb: 	MLP_units: 200
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 5
wandb: 	Patience: 20
wandb: 	Training_size: 991232
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense (Dense)               (None, 200)               1200      
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 batch_normalization_1 (Batc  (None, 200)              800       
 hNormalization)                                             

# Rolling window

In [37]:
def calculate_error(predictions, original):
    m = MeanSquaredError()
    m.update_state(predictions, original)
    print("MSE:", m.result().numpy())
    m = RootMeanSquaredError()
    m.update_state(predictions, original)
    print("RMSE:", m.result().numpy())

class config_object:
    def __init__(self, config):
        self.MLP_units = config['MLP_units']
        self.MLP_layers = config['MLP_layers']
        self.Bn_momentum = config['Bn_momentum']
        self.Lr = config['Lr']
        self.Lr_decay = config['Lr_decay']
        self.Minibatch_size = config['Minibatch_size']
        self.Min_delta = config['Min_delta']
        self.Patience = config['Patience']
        self.Num_features = config['Num_features']
        self.Architecture = config['Architecture']
        

In [38]:
num_windows = 99

config = {
    'MLP_units': 200,
    'MLP_layers': 3,
    'Bn_momentum': 0.4,
    'Lr': 0.0564,
    'Lr_decay': 0.809,
    'Leaky_relu_alpha': 0.1,
    'Minibatch_size': 4096,
    'Min_delta': 1,
    'Patience': 20,
    'Num_features': 5, 
    'Architecture': 'LSTM-MLP v1.0',
}

df_test_combined = pd.DataFrame()

if not checkpoint_time:
    checkpoint_time = datetime.now().strftime("%m-%d_%H-%M")
print(f'Checkpoint time: {checkpoint_time}')

for window in range(starting_window, num_windows):
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df, window_number=window)

    print('-------------------------------------------')
    print(f'Window {window + 1} of {num_windows}')
    print("Training start: ", train_start, "Validation start: ", val_start, "Test start: ", test_start)
        
    if google_colab:
        checkpoint_path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/{checkpoint_time}/{train_start}/'
    else:
        checkpoint_path = f'./checkpoints/{checkpoint_time}/{train_start}/'

    config['Dataset'] = f'{train_start} - {val_start} - {test_start}'

    if run_training:
        trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = config, project = 'Deep learning for option pricing - MLP rw', checkpoint_path = checkpoint_path)
    c_model = load_model(checkpoint_path + ".h5")
    
    # Make predictions
    predictions = c_model(test_x)

    print(f'--- Predictions for test_start {test_start} ---')
    calculate_error(predictions, test_y)
    print('-------------------------------------------')
    
    df_test["Prediction"] = predictions
    df_test_combined = pd.concat([df_test_combined, df_test[["Quote_date", "Price", "Prediction"] + bs_vars]])
    
    del train_x
    del train_y
    del val_x
    del val_y
    del test_x
    del test_y
    del train_start
    del val_start
    del test_start
    del df_test

print(f"--- All model predictions ---")
calculate_error(df_test_combined["Prediction"], df_test_combined["Price"])
print("-------------------------------------------")

if google_colab == False:
    predictions_path = './predictions/'
    if checkpoint_path and not os.path.exists(predictions_path):
        os.makedirs(predictions_path)
    df_test_combined.to_csv(f'{predictions_path}{checkpoint_time}.csv')
else:
  path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/06. Predictions/{checkpoint_time}.csv'
  df_test_combined.to_csv(path)

Checkpoint time: 05-24_16-50
--------------Dataframe dates--------------
Train: 2011-12-01 - 2014-11-28
Val: 2014-12-01 - 2014-12-31
Test: 2015-01-02 - 2015-01-30
-------------------------------------------
Train shape: (991232, 5),
Val shape: (61356, 5),
Test shape: (56797, 5),
-------------------------------------------
Window 1 of 99
Training start:  2011-12-01 Validation start:  2014-12-01 Test start:  2015-01-01


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense (Dense)               (None, 200)               1200      
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 batch_normalization_1 (Batc  (None, 200)              800       
 hNormalization)                                             

epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▄█▂▅▁▁▂▂▂█▃▁▂▂▂▂▂▂▄▂▂▁▃▄▃▂▆▁▂▂▁▂
best_epoch,11
best_val_loss,25.79076
epoch,31
loss,21.41548
val_loss,57.84241


--- Predictions for test_start 2015-01-01 ---
MSE: 16.829733
RMSE: 4.1024055
-------------------------------------------



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


--------------Dataframe dates--------------
Train: 2012-02-01 - 2015-01-30
Val: 2015-02-02 - 2015-02-27
Test: 2015-03-02 - 2015-03-31
-------------------------------------------
Train shape: (991232, 5),
Val shape: (57283, 5),
Test shape: (74992, 5),
-------------------------------------------
Window 3 of 99
Training start:  2012-02-01 Validation start:  2015-02-01 Test start:  2015-03-01


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense (Dense)               (None, 200)               1200      
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 batch_normalization_1 (Batc  (None, 200)              800       
 hNormalization)                                             

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▄▂▂▁▂▄▂▃▂▂▃▁▃▆▁▃▁▃
best_epoch,13
best_val_loss,8.25153
epoch,19
loss,27.00829
val_loss,22.7594


KeyboardInterrupt: 

In [ ]:
# Define a function to calculate RMSE
def calculate_rmse(group):
    rmse_model_1 = np.sqrt(mean_squared_error(group["Price"], group["Prediction"]))
    return pd.Series({"RMSE_Model_1": rmse_model_1})

# Group data by Quote_date and calculate RMSE for each group
rmse_df = df_test_combined.groupby("Quote_date").apply(calculate_rmse).reset_index()

# Plot the RMSE values for each model
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(data=rmse_df, x="Quote_date", y="RMSE_Model_1", label="Model", ax=ax)

# Set the interval for x-axis labels
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))

plt.xlabel("Quote_date")
plt.ylabel("RMSE")
plt.title("RMSE for Models")
plt.legend()
plt.xticks(rotation=45)
plt.show()